In [1]:
import numpy as np
from scipy.special import erf
import os
import sys
import time
import copy
import logging
import matplotlib.pyplot as plt

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)



from Dataset.flight_dataset_for_LS import load_flight_dataset_matrices_for_LS, save_flight_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b


from RP_mechanisms.optim_RP_DP import compute_largest_l2
from LS_mechanisms.alt19_LS import ALT19LS_mech
from LS_mechanisms.optim_LS_DP import LS_fromoptim_RP_mech, OptimalLS_mech
from LS_mechanisms.LS_accuracy_estimator import LS_accuracy_estimator

In [2]:
logfile_path = os.path.join(log_dir, 'LS-flight-default.log')
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.FileHandler(logfile_path),
        logging.StreamHandler()
    ]
)

def store_array_str(data_array):
    converted_str = np.array_str(data_array)
    converted_str = converted_str.replace(' ',',')
    return converted_str

file_X_path = os.path.join(project_dir, "Dataset", "flight-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "flight-LR-y.txt")

X = None
y = None
try:
    X, y = load_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    save_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_flight_dataset_matrices_for_LS()
    
B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [3]:
def generate_default_configuration():
    kwargs = {
        'database': A,
        'r' : 1200,
        'CI' : 0.95,
        'bootstrap_samples': 100, 
        'l2' : compute_largest_l2(A)
    }
    
    return kwargs

kwargs = generate_default_configuration()

In [4]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

0.8641385229788721

In [5]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-11-12 12:27:48,549 cost 2.6155 seconds


In [6]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-11-12 12:27:48,605 (bootstrap_result(lb_one_sided=np.float64(0.4606225016713969), lb_two_sided=np.float64(0.46035450272705875), ub_one_sided=np.float64(0.4699144212501735), ub_two_sided=np.float64(0.47148420553417403)), bootstrap_result(lb_one_sided=np.float64(1.82250362710563), lb_two_sided=np.float64(1.8191178961078092), ub_one_sided=np.float64(1.855671456280788), ub_two_sided=np.float64(1.8605011223241532)))
2025-11-12 12:27:48,606 cost 0.0347 seconds


In [7]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: invalid value encountered in scalar multiply
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)


np.float64(0.7452001524823914)

In [8]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-11-12 12:52:13,953 cost 1465.3263 seconds


In [9]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-11-12 12:52:14,006 (bootstrap_result(lb_one_sided=np.float64(0.39334163994515675), lb_two_sided=np.float64(0.39314676668066995), ub_one_sided=np.float64(0.4010155931271154), ub_two_sided=np.float64(0.4016755392326422)), bootstrap_result(lb_one_sided=np.float64(1.6213811153652622), lb_two_sided=np.float64(1.620151913268019), ub_one_sided=np.float64(1.6412651347723624), ub_two_sided=np.float64(1.6416273664342966)))
2025-11-12 12:52:14,006 cost 0.0411 seconds


In [10]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.compute_constant(epsilon, delta)

np.float64(2.0599687150443087)

In [11]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-11-12 13:32:46,734 cost 2431.8648 seconds


In [12]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-11-12 13:32:46,806 (bootstrap_result(lb_one_sided=np.float64(0.8246889682525818), lb_two_sided=np.float64(0.824040036222986), ub_one_sided=np.float64(0.8318516497819459), ub_two_sided=np.float64(0.8332473662206419)), bootstrap_result(lb_one_sided=np.float64(5.496763779743661), lb_two_sided=np.float64(5.492214830762541), ub_one_sided=np.float64(5.720464415003367), ub_two_sided=np.float64(5.733842194162772)))
2025-11-12 13:32:46,807 cost 0.0487 seconds
